# Logistic Regression

我們將透過這份作業學習 Logistic Regression。

## 1 - Packages ##

首先，我們需要 import 一些需要使用到的套件。
- [numpy](http://www.numpy.org) 是 Python 的擴充程式庫，提供大量的數學函式庫與矩陣運算等。
- [Pandas](http://pandas.pydata.org) 是一個提供數據分析的 Python 函式庫。
- [Scikit-learn](http://scikit-learn.org) 是一個提供 Machine Learning 運算的函式庫。

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model

## 2 - Overview of the Dataset
我們有一個 Dataset 包含
- X
- Y1
- Y2
- Y3

我們之後會透過 Scikit-learn 來找出他們之間的關係。

首先，我們透過 `pd.read_csv()` 來讀取 csv 檔案。

In [2]:
# csv 檔案的位置
csv_file = "logistic.regression.csv"

# 讀取 csv
dataset = pd.read_csv(csv_file)

`dataset`是一個 DataFrame，我們可以使用 `print()` 來看看 dataset。

In [3]:
# 印出 dataset
print(dataset)

      X        Y1        Y2            Y3
0  -3.0  0.002473  0.000006  1.523000e-08
1  -2.9  0.003018  0.000009  2.775080e-08
2  -2.8  0.003684  0.000014  5.056530e-08
3  -2.7  0.004496  0.000020  9.213600e-08
4  -2.6  0.005486  0.000030  1.678830e-07
5  -2.5  0.006693  0.000045  3.059020e-07
6  -2.4  0.008163  0.000068  5.573900e-07
7  -2.3  0.009952  0.000101  1.015630e-06
8  -2.2  0.012128  0.000151  1.850600e-06
9  -2.1  0.014774  0.000225  3.372000e-06
10 -2.0  0.017986  0.000335  6.144170e-06
11 -1.9  0.021881  0.000500  1.119540e-05
12 -1.8  0.026597  0.000746  2.039910e-05
13 -1.7  0.032295  0.001113  3.716890e-05
14 -1.6  0.039166  0.001659  6.772410e-05
15 -1.5  0.047426  0.002473  1.233950e-04
16 -1.4  0.057324  0.003684  2.248170e-04
17 -1.3  0.069138  0.005486  4.095670e-04
18 -1.2  0.083173  0.008163  7.460290e-04
19 -1.1  0.099750  0.012128  1.358520e-03
20 -1.0  0.119203  0.017986  2.472623e-03
21 -0.9  0.141851  0.026597  4.496273e-03
22 -0.8  0.167982  0.039166  8.162

我們分別取出 `data_X`, `data_Y1`, `data_Y2`, `data_Y3`。
- 我們可以使用 [`iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html)取得數據。

In [4]:
# 取出 X
data_X = dataset.iloc[:, 0]

# 取出 Y1
data_Y1 = dataset.iloc[:, 1]

# 取出 Y2
data_Y2 = dataset.iloc[:, 2]

# 取出 Y3
data_Y3 = dataset.iloc[:, 3]

我們可以觀察這些變數的 `type` 和 `shape`:

In [5]:
print("data_X type:"+ str(type(data_X)))
print("data_X shape:"+ str(data_X.shape))
print("data_Y1 type:"+ str(type(data_Y1)))
print("data_Y1 shape:"+ str(data_Y1.shape))
print("data_Y2 type:"+ str(type(data_Y2)))
print("data_Y2 shape:"+ str(data_Y2.shape))
print("data_Y3 type:"+ str(type(data_Y3)))
print("data_Y3 shape:"+ str(data_Y3.shape))

data_X type:<class 'pandas.core.series.Series'>
data_X shape:(61,)
data_Y1 type:<class 'pandas.core.series.Series'>
data_Y1 shape:(61,)
data_Y2 type:<class 'pandas.core.series.Series'>
data_Y2 shape:(61,)
data_Y3 type:<class 'pandas.core.series.Series'>
data_Y3 shape:(61,)




我們發現四個變數的 shape 都是 **(300,)** , 使用 `np.array()` 轉成 array 後也會是 **(300,)** ，我們稱之為 **rank 1 array**。

在 Andrew Ng 的 Neural Networks and Deep Learning 課程中，有提到我們**應當避免使用 rank 1 array**。

這裡我們可以使用`data[:, np.newaxis]`或是`data[:, None]`來解決此問題。


In [6]:
# Reshape Rank 1 array
data_X  = data_X[:,None]
data_Y1 = data_Y1[:,None]
data_Y2 = data_Y2[:,None]
data_Y3 = data_Y3[:,None]

我們可以再次觀察這些變數的 `type` 和 `shape`:

In [7]:
print("data_X type:"+ str(type(data_X)))
print("data_X shape:"+ str(data_X.shape))
print("data_Y1 type:"+ str(type(data_Y1)))
print("data_Y1 shape:"+ str(data_Y1.shape))
print("data_Y2 type:"+ str(type(data_Y2)))
print("data_Y2 shape:"+ str(data_Y2.shape))
print("data_Y3 type:"+ str(type(data_Y3)))
print("data_Y3 shape:"+ str(data_Y3.shape))

data_X type:<class 'numpy.ndarray'>
data_X shape:(61, 1)
data_Y1 type:<class 'numpy.ndarray'>
data_Y1 shape:(61, 1)
data_Y2 type:<class 'numpy.ndarray'>
data_Y2 shape:(61, 1)
data_Y3 type:<class 'numpy.ndarray'>
data_Y3 shape:(61, 1)


另外，我們需要對`data_Y`的值進行一些改變，當資料`>=0.5`時，將其換成1；當資料`<0.5`時，將其換成0。

In [8]:
data_Y1 = np.where(data_Y1>0.5, 1, 0)
data_Y2 = np.where(data_Y2>0.5, 1, 0)
data_Y3 = np.where(data_Y3>0.5, 1, 0)

我們可以使用 `print()` 來看看 `data_Y1`。

In [9]:
print(data_Y1)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


## 3 - Choose a model
在這次的作業中，我們選擇使用 `Scikit-learn` 中的 [`LogisticRegression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 來建立模型。

In [10]:
# 建立 Linear Regression 模型
clf = linear_model.LogisticRegression(C=1e5, solver='lbfgs')

## 4 - Train the model
建立好模型之後，我們可以使用 `fit()` 來訓練模型。

### 4-1 Example 1

In [11]:
# 訓練模型
clf.fit(data_X,data_Y1.ravel())

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

最後我們可以列印出 $Coefficient$ 和 $Intercept$ 的值。

In [12]:
# 列印 Coefficient 和 Intercept
print("Coefficient:"+ str(clf.coef_))
print("Intercept:"+ str(clf.intercept_))

Coefficient:[[93.09817642]]
Intercept:[-4.65813391]
